# Using Asteroid

This notebook was inspired by Andrew Shitov's excellent book [Using Raku: 100 Programming Challenges Solved with the Brand-New Raku Programming Language](https://andrewshitov.com/wp-content/uploads/2020/01/Using-Raku.pdf).  Here of course we use Asteroid to solve these programming challenges.

In [1]:
# make the Asteroid interpreter available in this notebook
from asteroid_interp import interp

**Note**: We use program as strings in this notebook so that we can run them right here in this notebook.

# Part I
# Chapter 1: Strings
## 1.1 Using Strings


### 1. Hello, World!

> Print ‘Hello, World!’

The canonical `Hello, World!` program.  The easiest way to write this in Asteroid is,

In [2]:
program = \
'''
load "io".
println "Hello, World!".
'''
interp(program)

Hello, World!


Two other print functions exists: 
- `print` - print without appending a new line character.
- `raw_print` - print internal term structure. 

In [3]:
program = \
'''
load "io".
raw_print "Hello, World!".
'''
interp(program)

('string', 'Hello, World!')


Here we can see that an Asteroid string is tuple consisting of a type field and a value field.

### 2. Greet a person

> Ask a user for their name and greet them by printing ‘Hello, <Name\>!’

Here is our first solution using a separate function for each of the steps,   

In [4]:
program = \
'''
load "io".
print ("Enter your name: ").
let name = input().
print ("Hello, "+name+"!").
'''
interp(program)

Enter your name: Leo
Hello, Leo!

Letting the function `input` do the prompting,

In [5]:
program = \
'''
load "io".
let name = input("Enter your name: ").
print ("Hello, "+name+"!").
'''
interp(program)

Enter your name: Leo
Hello, Leo!

Doing everything in one step,

In [6]:
program = \
'''
load "io".
print ("Hello, "+input("Enter your name: ")+"!").
'''
interp(program)

Enter your name: Leo
Hello, Leo!

### 3. String length

> Print the length of a string.

In order to print the length of a string we can use the function `len` available in the `util` module,

In [7]:
program = \
'''
load "io".
load "util".
println (len("Hello!")).
'''
interp(program)

6


We can also use the string member function `length` in order to compute the length of the string,

In [8]:
program = \
'''
load "io".
println ("Hello!" @length()).
'''
interp(program)

6


### 4. Unique digits

> Print unique digits from a given integer number.

In order to accomplish this we take advantage of the string `explode` function and the `sort` function on lists.
Finally we use the `reduce` function to map a list with repeated digits to a list with unique digits,

In [4]:
program = \
'''
load "io".

function unique with (x,y) do
    if not (x @member(y)) do
        return x @append(y).
    else do
        return x.
    end
end
        
let digits = (("332211" @explode()) @sort()) @reduce(unique,[]).
println digits.
assert(digits == ["1","2","3"]).
'''
interp(program)

[1,2,3]


Probably the most noteworthy characteric about this program is the `reduce` function.  The `reduce` function applies a binary function to a list.  The first argument of the binary function acts like an accumulator and the second argument get instantiated with the elements of the list to be processed.  In our function `unique` the variable `x` is the accumulator with an initial value of `[]`.  The function tests whether the element `y` is on the list.  If it is not then it adds it to the list otherwise it just returns the accumulator unchanged.

## 1.2 Modifying string data

### 5. Reverse a string

> Print a string in the reversed order from right to left.

We use the `explode` function to turn a string into a list of characters, then we reverse the list and turn it back into a string using the `join` function,

In [10]:
program = \
'''
load "io".
let str = (("Hello, World!" @explode()) @reverse()) @join("").
println str.
assert(str == "!dlroW ,olleH").
'''
interp(program)

!dlroW ,olleH


### 6. Removing blanks from a string

> Remove leading, trailing and double spaces from a given string.

In [11]:
program = \
'''
load "io".
let str = ("   Hello  ,   World    !   " @trim()) @replace("  ","").
println str.
assert(str == "Hello, World!").
'''
interp(program)

Hello, World!


### 7. Camel case

> Create a camel-case identifier from a given phrase.

In this task, we will form the `CamelCase` variable names from a given phrase.
Names created in this style are built of several words; each of which starts
with a capital letter.

In [12]:
program = \
'''
load "io".

function title with w do
    let letter_list = (w @tolower()) @explode().
    let first_letter = (letter_list@0) @toupper().
    if letter_list @length() > 1 do
        let title_case = ([first_letter]+letter_list@[1 to letter_list@length()-1]) @join("").
    else
        let title_case = first_letter.
    end
    return title_case.
end

let str = "once upon a time".
let camel_str = ((str @split()) @map(title)) @join("").
println camel_str.
assert(camel_str == "OnceUponATime").
'''
interp(program)

OnceUponATime


### 8. Incrementing filenames

> Generate a list of filenames like file1.txt, file2.txt, etc.

In [15]:
program = \
'''
load "io".

let root = "file".
let ext = ".txt".

for i in 1 to 5 do
    println (root+i+ext).
end
'''
interp(program)

file1.txt
file2.txt
file3.txt
file4.txt
file5.txt


### 9. Random passwords

> Generate a random string that can be used as a password.

In our solution we take advantage of Asteroid's `Pick` object.  The `Pick` object maintains a list of items that we can randomly select from using the `pick` member function.  As input to the `Pick` object we compute a bunch of lists of characters that are useful for password construction.  The function `achar` converts a decimal ASCII code to a single character string.

In [42]:
program = \
'''
load "io".
load "util".

-- make up lists of symbols useful for password construction
let int_list = [0 to 9] @map(tostring).
let lc_list = [97 to 122] @map(achar). -- lower case characters
let uc_list = [65 to 90] @map(achar). --upper case characters
let sp_list = ["!","_","#","$","%","*"].
-- build the overall pick list of symbols
let pick_list = int_list+lc_list+uc_list+sp_list.

-- generate the password and print it.
println ((Pick pick_list) @pick(15)) @join("").
'''
interp(program)

4CUPtz_l0ANaohy


### 10. DNA-to-RNA transcription

> Convert the given DNA sequence to a compliment RNA.

We’ll not dig deep into the biology aspect of the problem. For us, it is important that the DNA is a string containing the four letters A, C, G, and T,
and the RNA is a string of A, C, G, and U. The transformation from DNA
to RNA happens according to the following table:
```
DNA: A C G T
RNA: U G C A
```
We will solve this programming problem using Asteroid's first-class patterns. We could have solved this with just testing equality on DNA characters but using first-class patterns in more general and can be applied to problems with a more structured mapping relationship.

In [47]:
program = \
'''
load "io".
load "util".

let dna2rna_table = 
    [
    ("A","U"),
    ("C","G"),
    ("G","C"),
    ("T","A")
    ].

function dna2rna with x do
    for (dna,rna) in dna2rna_table do
        if x is *dna do
            return rna.
        end
    end
    throw Error("unknown dna char "+x).
end

let dna_seq = "ACCATCAGTC".
let rna_seq = ((dna_seq @explode()) @map(dna2rna)) @join("").
println rna_seq.
assert(rna_seq == "UGGUAGUCAG").
'''
interp(program)

UGGUAGUCAG


### 11. Caesar cipher

> Encode a message using the Caesar cipher technique.

The Caesar code is a simple method of transcoding the letters of the message
so that each letter is replaced with the letter that occurs in the alphabet N
positions earlier or later.
For example, if N is 4, then the letter e becomes a, f is transformed to b,
etc. The alphabet is looped so that z becomes v, and letters a to d become
w to z.

In [75]:
program = \
'''
load "io".
load "util".

let encode_table = [119 to 122] @map(achar) + [97 to 118] @map(achar).

function encode with (v:%string) %if len(v) == 1 do
    if not (ascii(v) in [97 to 122]) do
        return v.
    else 
        return encode_table @(ascii(v)-ascii("a")).
    end
end

function decode with (v:%string) %if len(v) == 1 do
    if not (ascii(v) in [97 to 122]) do
        return v.
    else 
        return encode_table @(ascii(v)-ascii("w")+4).
    end
end

let message = "hello, world!"
let secret = ((message @explode()) @map(encode)) @join("").
println secret.

let decoded_msg = ((secret @explode()) @map(decode)) @join("").
println decoded_msg.
'''
interp(program)

dahhk, sknhz!
hello, world!
